# Import Required Libraries
Import the necessary libraries, such as requests, for making API calls.

In [8]:
# Import the necessary library for making API calls
#%pip install requests
#%pip install pandas
#%pip install datetime
#%pip install os
#%pip install cudf
#%pip install cupy
#%pip install openpyxl
#%pip install textblob
#%pip install transformers
#%pip install torch
import requests
import pandas as pd
import datetime as dt
import os
from textblob import TextBlob
from transformers import pipeline

c:\Users\ravit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define API Key
Define the API key

In [4]:
API_Eodhd = "66c0aeb1357b15.87356825"

# Get list of tickets under an exchange
Define a function get list of tickers under an exchange

In [5]:
def fetch_exchanges_list(api_token=API_Eodhd, filtered_columns=["Name", "Code", "Country", "Currency"]):
    """
    Fetch the list of exchanges from the EODHD API and return it as a filtered Pandas DataFrame.

    Parameters:
        api_token (str): Your API token for authentication.
        filtered_columns (list): List of columns to include in the output DataFrame.

    Returns:
        pd.DataFrame: The exchange list as a Pandas DataFrame with selected columns.
    """
    url = f'https://eodhd.com/api/exchanges-list/?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    exchanges_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if filtered_columns:

        exchanges_df = exchanges_df[filtered_columns]
    
    return exchanges_df

def fetch_exchange_symbols(exchange_code, api_token=API_Eodhd, filtered_columns=None):
    """
    Fetch the symbol list for a given exchange from the EODHD API and return it as a Pandas DataFrame.

    Parameters:
        exchange_code (str): The code of the exchange (e.g., "US").
        api_token (str): Your API token for authentication.
        filtered_columns (list): List of columns to include in the output DataFrame (optional).

    Returns:
        pd.DataFrame: The symbol list as a Pandas DataFrame with selected columns.
    """
    url = f'https://eodhd.com/api/exchange-symbol-list/{exchange_code}?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    symbols_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if filtered_columns:
        symbols_df = symbols_df[filtered_columns]
    
    return symbols_df

tickers = fetch_exchange_symbols("AS")
print(tickers)

      Code                                               Name      Country  \
0    1ARKG   LS ARK Genomic Revolution Tracker ETP Securities  Netherlands   
1    1ARKK                                  ARK INNOVATION 1X  Netherlands   
2    1ARKW                LS ARK NextGen Internet Tracker ETP  Netherlands   
3     2BRK                                  2x Long Berkshire  Netherlands   
4     2HCA  iShares MSCI World Health Care Sector ESG UCIT...  Netherlands   
..     ...                                                ...          ...   
429   WSRI  Amundi MSCI World SRI Climate Net Zero Ambitio...  Netherlands   
430   WTCH  SPDR® MSCI World Technology UCITS ETF USD Acc EUR  Netherlands   
431   WTEL  SPDR® MSCI World Communication Services UCITS ...  Netherlands   
432   WUTI       SPDR® MSCI World Utilities UCITS ETF USD Acc  Netherlands   
433   WVAL                    SPDR MSCI World Value UCITS ETF  Netherlands   

    Exchange Currency Type          Isin  
0         AS      EU

# Get historical market data
uses a function to get historical market data

In [6]:
def Fetch_historical_price(stock1, begin_date, end_date, period, api_token=API_Eodhd, columns=["adjusted_close"]):
    """
    Fetch stock data from the EODHD API and return it as a filtered Pandas DataFrame.

    Parameters:
        stock1 (str): The stock symbol (e.g., "MCD.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        period (str): The period (e.g., "d" for daily).
        api_token (str): Your API token for authentication.
        columns (list): List of columns to filter (e.g., ['open', 'close','adjusted_close',volume]).

    Returns:
        pd.DataFrame: The filtered stock data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/eod/{stock1}?from={begin_date}&to={end_date}&period={period}&api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON to Pandas DataFrame
    pandas_df = pd.DataFrame(response)
    
    # Filter columns if specified
    if columns:
       filtered_columns = ['date'] + columns
       pandas_df = pandas_df[filtered_columns]
    
    return pandas_df


ZYXI = Fetch_historical_price("ZYXI.US", "2023-01-01", "2023-10-01", "d" )
print(ZYXI)

           date  adjusted_close
0    2023-01-03           13.88
1    2023-01-04           14.11
2    2023-01-05           14.14
3    2023-01-06           15.65
4    2023-01-09           15.65
..          ...             ...
182  2023-09-25            8.41
183  2023-09-26            8.25
184  2023-09-27            8.23
185  2023-09-28            8.25
186  2023-09-29            8.00

[187 rows x 2 columns]


# Getting financial news
Creates a function to retrive financial news

In [7]:
def fetch_news_data(stock, tag ,begin_date, end_date, offset =0 , api_token =API_Eodhd):
    """
    Fetch news data for a given stock and return it as a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        begin_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        tag (str): The tag to filter news (e.g., "balance sheet"). List of tags : 'balance sheet', 'capital employed', 'class action', 'company announcement', 'consensus eps estimate', 'consensus estimate', 'credit rating', 'discounted cash flow', 'dividend payments', 'earnings estimate', 'earnings growth', 'earnings per share', 'earnings release', 'earnings report', 'earnings results', 'earnings surprise', 'estimate revisions', 'european regulatory news', 'financial results', 'fourth quarter', 'free cash flow', 'future cash flows', 'growth rate', 'initial public offering', 'insider ownership', 'insider transactions', 'institutional investors', 'institutional ownership', 'intrinsic value', 'market research reports', 'net income', 'operating income', 'present value', 'press releases', 'price target', 'quarterly earnings', 'quarterly results', 'ratings', 'research analysis and reports', 'return on equity', 'revenue estimates', 'revenue growth', 'roce', 'roe', 'share price', 'shareholder rights', 'shareholder', 'shares outstanding', 'split', 'strong buy', 'total revenue', 'zacks investment research', 'zacks rank'
        offset (int): The offset for pagination.
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The news data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/news?s={stock}&t={tag}&offset={offset}&limit=10&api_token={api_token}&from={begin_date}&to={end_date}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON response to Pandas DataFrame
    news_df = pd.DataFrame(response)
    
    return news_df

# Example usage
stock = "AAPL.US"
begin_date = "2023-01-01"
end_date = "2023-10-01"
tag = "balance sheet"
offset = 0

news_df = fetch_news_data(stock, tag ,begin_date, end_date)
print(news_df)

                        date  \
0  2023-03-24T13:00:11+00:00   
1  2023-02-17T11:01:03+00:00   
2  2023-02-14T14:24:02+00:00   
3  2023-01-26T13:06:00+00:00   
4  2023-01-18T11:00:49+00:00   
5  2023-01-14T12:04:45+00:00   
6  2023-01-01T23:08:43+00:00   

                                               title  \
0  Apple (NASDAQ:AAPL) Seems To Use Debt Quite Se...   
1        Is Apple (NASDAQ:AAPL) Using Too Much Debt?   
2  This Semiconductor Stock Has Too Much Dependen...   
3  If Cash Is King, These Nasdaq Stocks Reign Sup...   
4  Apple (NASDAQ:AAPL) Could Easily Take On More ...   
5            7 Stocks to Buy and Hold For a Lifetime   
6  Lessons Learned From Visa and Mastercard in a ...   

                                             content  \
0  Howard Marks put it nicely when he said that, ...   
1  David Iben put it well when he said, 'Volatili...   
2  In today's video, Jose Najarro, Nick Rossolill...   
3  Because of that, companies that generate lots ...   
4  Howard Mark

# Getting fundamental data
Creating a function retrive fundamental data

In [8]:
def fetch_fundamentals(stock, api_token=API_Eodhd):
    """
    Fetch fundamental data for a given stock and return it as a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The fundamental data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/fundamentals/{stock}?api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Flatten the JSON response and convert it to a Pandas DataFrame
    fundamentals_df = pd.json_normalize(response)
    
    return fundamentals_df

# Example usage
api_token = API_Eodhd  # Replace with your actual API token
stock = "AAPL.US"

fundamentals_df = fetch_fundamentals(stock, api_token)
print(fundamentals_df)

  General.Code  General.Type General.Name General.Exchange  \
0         AAPL  Common Stock    Apple Inc           NASDAQ   

  General.CurrencyCode General.CurrencyName General.CurrencySymbol  \
0                  USD            US Dollar                      $   

  General.CountryName General.CountryISO General.OpenFigi  ...  \
0                 USA                 US     BBG000B9XRY4  ...   

  Financials.Income_Statement.yearly.1985-09-30.otherItems  \
0                                               None         

  Financials.Income_Statement.yearly.1985-09-30.incomeTaxExpense  \
0                                        58800000.00               

  Financials.Income_Statement.yearly.1985-09-30.totalRevenue  \
0                                      1918300000.00           

  Financials.Income_Statement.yearly.1985-09-30.totalOperatingExpenses  \
0                                       695000000.00                     

  Financials.Income_Statement.yearly.1985-09-30.costOfRevenue

# Getting macro economic data
Creating a function retrive macroeconomic data

In [9]:
def fetch_macro_indicator(country, indicator, api_token=API_Eodhd):
    """
    Fetch macroeconomic indicator data for a given country and return it as a Pandas DataFrame.

    Parameters:
        country (str): The country code (e.g., "USA").
        indicator (str): The macroeconomic indicator (e.g., "inflation_consumer_prices_annual"). List of indicators: https://eodhd.com/financial-apis/macroeconomics-data-and-macro-indicators-api
        api_token (str): Your API token for authentication.

    Returns:
        pd.DataFrame: The macroeconomic indicator data as a Pandas DataFrame.
    """
    url = f'https://eodhd.com/api/macro-indicator/{country}?indicator={indicator}&api_token={api_token}&fmt=json'
    response = requests.get(url).json()
    
    # Convert JSON response to Pandas DataFrame
    macro_df = pd.DataFrame(response)
    
    return macro_df

# Example usage
country = "USA"
indicator = "inflation_consumer_prices_annual"
macro_df = fetch_macro_indicator(country, indicator)
print(macro_df)


   CountryCode    CountryName                              Indicator  \
0          USA  United States  Inflation, consumer prices (annual %)   
1          USA  United States  Inflation, consumer prices (annual %)   
2          USA  United States  Inflation, consumer prices (annual %)   
3          USA  United States  Inflation, consumer prices (annual %)   
4          USA  United States  Inflation, consumer prices (annual %)   
..         ...            ...                                    ...   
59         USA  United States  Inflation, consumer prices (annual %)   
60         USA  United States  Inflation, consumer prices (annual %)   
61         USA  United States  Inflation, consumer prices (annual %)   
62         USA  United States  Inflation, consumer prices (annual %)   
63         USA  United States  Inflation, consumer prices (annual %)   

          Date  Period   Value  
0   2023-12-31  Annual  4.1163  
1   2022-12-31  Annual  8.0028  
2   2021-12-31  Annual  4.6979  
3  

# Getting intraday stock data
Creating a function retrive stock data

In [10]:
def fetch_intraday_data(stock, api_token= API_Eodhd):
    """
    Fetch intraday data for a given stock from the EODHD API and convert it to a Pandas DataFrame.

    Parameters:
        stock (str): The stock symbol (e.g., "AAPL.US").
        api_token (str): Your API token for authentication.
        interval (str): The interval for intraday data (e.g., "1m", "5m", "15m", "1h"). Default is "1m".

    Returns:
        pd.DataFrame: A DataFrame containing the intraday data.
    """
    url = f'https://eodhd.com/api/intraday/{stock}?api_token={api_token}&fmt=json'
    response = requests.get(url)
    
    # Parse the JSON response
    data = response.json()
    
    # Convert JSON to Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the 'datetime' column is in datetime format
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
    
    return df

# Example usage
stock = "AAPL.US"

intraday_data = fetch_intraday_data(stock)
print(intraday_data.head())

    timestamp  gmtoffset            datetime        open        high  \
0  1733236200          0 2024-12-03 14:30:00  239.192398  239.940002   
1  1733236500          0 2024-12-03 14:35:00  239.550003  239.668106   
2  1733236800          0 2024-12-03 14:40:00  239.389999  239.710006   
3  1733237100          0 2024-12-03 14:45:00  239.610000  240.639801   
4  1733237400          0 2024-12-03 14:50:00  240.619995  240.729995   

          low       close     volume  
0  238.900299  239.565002  2496034.0  
1  238.990005  239.369995   532222.0  
2  239.110000  239.615005   414728.0  
3  239.589996  240.589996   868554.0  
4  240.270004  240.695007   571623.0  


# Running correlation between all the stocks in the exchange
Creates a list of all the correlations betweens stocks and sorts them

In [10]:
def calculate_and_sort_correlations(tickers_df, exchange, instrument_type="Common Stock", api_token=API_Eodhd, start_date="2025-01-01", end_date="2025-03-30", period="d"):
    """
    Calculate the correlation between all stocks in an exchange and sort them from least to highest.

    Parameters:
        tickers_df (pd.DataFrame): DataFrame containing stock tickers.
        exchange (str): Exchange code to append to tickers (e.g., "US").
        instrument_type (str): Type of the instrument to filter (e.g., "Common Stock").
        api_token (str): API token for authentication.
        start_date (str): Start date for historical data.
        end_date (str): End date for historical data.
        period (str): Period for historical data (e.g., "d" for daily).

    Returns:
        pd.DataFrame: DataFrame containing stock pairs and their correlation, sorted by correlation value.
    """
    # Filter tickers to include only the specified instrument type
    filtered_tickers_df = tickers_df[tickers_df["Type"] == instrument_type]

    stock_data = {}
    for ticker in filtered_tickers_df['Code']:
        try:
            # Combine ticker with exchange code
            full_ticker = f"{ticker}.{exchange}"
            stock_df = Fetch_historical_price(full_ticker, start_date, end_date, period, api_token, columns=["adjusted_close"])
            stock_data[ticker] = stock_df.set_index("date")["adjusted_close"]
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    # Combine all stock data into a single DataFrame
    combined_df = pd.DataFrame(stock_data)

    # Calculate correlation matrix
    correlation_matrix = combined_df.corr()

    # Create a list of all pairs and their correlations
    correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            stock1 = correlation_matrix.columns[i]
            stock2 = correlation_matrix.columns[j]
            correlation = correlation_matrix.iloc[i, j]
            correlations.append({"Instrument 1": stock1, "Instrument 2": stock2, "Correlation": correlation})

    # Convert the list of correlations to a DataFrame
    correlations_df = pd.DataFrame(correlations)

    # Sort the DataFrame by correlation value
    correlations_df = correlations_df.sort_values(by="Correlation", ascending=True).reset_index(drop=True)

    return correlations_df

# Example usage
tickers = fetch_exchange_symbols("AS")
tickers_subset = tickers
sorted_correlations_df = calculate_and_sort_correlations(tickers_subset, "AS", instrument_type="ETF")
print(sorted_correlations_df)

KeyboardInterrupt: 

# Exporting output to excel
Creates an excel export in the python folder

In [ ]:
# Define the output directory and file path
output_dir = r"E:\Business NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
output_file = os.path.join(output_dir, "sorted_correlations.xlsx")

# Export the DataFrame to an Excel file
sorted_correlations_df.to_excel(output_file, index=False)

print(f"Sorted correlations exported to {output_file}")

# Analyzing sentement with finbert model
Creates a function to use finbert model to analyze sentement

In [20]:

def analyze_sentiment_with_finbert(news_df):
    """
    Analyze the sentiment of news articles using FinBERT.

    Parameters:
        news_df (pd.DataFrame): DataFrame containing news articles.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Sentiment' column.
    """
    if 'content' not in news_df.columns:
        raise ValueError("The DataFrame must contain a 'content' column for sentiment analysis.")
    
    # Load the FinBERT sentiment analysis pipeline with explicit truncation
    finbert = pipeline(
        "sentiment-analysis",
        model="yiyanghkust/finbert-tone",
        tokenizer="yiyanghkust/finbert-tone",
        device=0,  # Use CPU (-1) or GPU (0 or higher)
        truncation=True,
        max_length=512  # Explicitly set the maximum length
    )
    
    # Apply sentiment analysis to the 'content' column
    def analyze_text(text):
        try:
            return finbert(text[:512])[0]['label']  # Truncate text to 512 characters
        except Exception as e:
            return f"Error: {e}"
    
    news_df['Sentiment'] = news_df['content'].apply(analyze_text)
    
    return news_df

# Example usage
api_token = API_Eodhd  # Replace with your actual API token
stock = "AAPL.US"
begin_date = "2023-01-01"
end_date = "2025-03-01"
tag = "balance sheet"

# Fetch news data
news_df = fetch_news_data(stock, tag, begin_date, end_date, api_token=api_token)

# Perform sentiment analysis with FinBERT
news_with_sentiment = analyze_sentiment_with_finbert(news_df)

print(news_with_sentiment)

Device set to use cpu


                        date  \
0  2023-03-24T13:00:11+00:00   
1  2023-02-17T11:01:03+00:00   
2  2023-02-14T14:24:02+00:00   
3  2023-01-26T13:06:00+00:00   
4  2023-01-18T11:00:49+00:00   
5  2023-01-14T12:04:45+00:00   
6  2023-01-01T23:08:43+00:00   

                                               title  \
0  Apple (NASDAQ:AAPL) Seems To Use Debt Quite Se...   
1        Is Apple (NASDAQ:AAPL) Using Too Much Debt?   
2  This Semiconductor Stock Has Too Much Dependen...   
3  If Cash Is King, These Nasdaq Stocks Reign Sup...   
4  Apple (NASDAQ:AAPL) Could Easily Take On More ...   
5            7 Stocks to Buy and Hold For a Lifetime   
6  Lessons Learned From Visa and Mastercard in a ...   

                                             content  \
0  Howard Marks put it nicely when he said that, ...   
1  David Iben put it well when he said, 'Volatili...   
2  In today's video, Jose Najarro, Nick Rossolill...   
3  Because of that, companies that generate lots ...   
4  Howard Mark